In [1]:
import importlib
from tqdm import tqdm
import json
calculate_doc_term_recall = importlib.import_module('train-deep-ct').calculate_doc_term_recall
calculate_qtokens = importlib.import_module('train-deep-ct').calculate_qtokens

!nosetests
from util import clean_qtokens

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [2]:
DIR = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/deep-ct/'
anchorText = {}

try:
    anchorText = clean_qtokens(json.load(open(DIR + 'ms-marco-orcas-deepct-qtokens.json', 'r')))
except Exception as e:
    print(e)
    print('Comment the following lines to create ms-marco-orcas-deepct-qtokens.json.')
    #with open(DIR + 'msmarco-document-orcas.jsonl') as f:
    #    for l in tqdm(f):
    #        l = json.loads(l)
    #        for target_doc in l['target_document']:
    #            if target_doc not in anchorText:
    #                anchorText[target_doc] = []
    #            anchorText[target_doc] += [l['contents']]
    #
    #for target_doc in tqdm([i for i in anchorText.keys()]):
    #    anchorText[target_doc] = calculate_qtokens(anchorText[target_doc])
    #json.dump(anchorText, open(DIR + 'ms-marco-orcas-deepct-qtokens.json', 'w'))

From qtokens with 1147461 documents, I filtered terms that may cause train/test overlap so that I now have 876950 documents.


In [3]:
len(anchorText)

876950

In [4]:
with open(DIR + 'msmarco-document-passages.jsonl') as docs, open(DIR + 'deep-ct-training-data-orcas-test-overlap-removed.jsonl', 'w') as out_file:
    for doc in tqdm(docs):
        doc = json.loads(doc)
        doc_id = doc['id']
        if doc_id not in anchorText:
            continue
        for passage_id, passage in doc['passages']:
            for i in calculate_doc_term_recall([{"doc":{"title": passage}}], anchorText[doc_id]):
                i['doc']['position'] = passage_id
                i['doc']['id'] = doc_id + '___' + str(passage_id)
                out_file.write(json.dumps(i) + '\n')

3213835it [3:55:05, 227.84it/s]


In [5]:
saved_docs = []
examples = 0

with open(DIR + 'deep-ct-training-data-orcas-test-overlap-removed.jsonl', 'r') as docs:
    for l in docs:
        l = json.loads(l)
        examples += 1
        saved_docs += [l['doc']['id'].split('___')[0]]

print('Examples: ', examples)
print('Covered Documents: ', len(set(saved_docs)))

Examples:  8171433
Covered Documents:  876950


In [6]:
docs_to_passage_count = {}

with open(DIR + 'deep-ct-training-data-orcas-test-overlap-removed.jsonl', 'r') as docs:
    for l in tqdm(docs):
        l = json.loads(l)
        doc_id = l['doc']['id'].split('___')[0]
        if doc_id not in docs_to_passage_count:
            docs_to_passage_count[doc_id] = 0
        docs_to_passage_count[doc_id] = docs_to_passage_count[doc_id] + 1


8171433it [01:15, 108632.85it/s]


In [19]:
from random import shuffle
selected_ids = []
count = 0
doc_ids = [i for i in docs_to_passage_count.keys()]
shuffle(doc_ids)

for doc_id in doc_ids:
    count += docs_to_passage_count[doc_id]
    selected_ids += [doc_id]
    if count > 2079858:
        break

selected_ids = set(selected_ids)
print('Have ', len(selected_ids), ' Ids')
print('Have ', count, ' Examples')

Have  224149  Ids
Have  2080050  Examples


In [20]:
with open(DIR + 'deep-ct-training-data-orcas-test-overlap-removed.jsonl', 'r') as docs, open(DIR + 'deep-ct-training-data-orcas-sampled-test-overlap-removed.jsonl', 'w') as out_file:
    for l in tqdm(docs):
        doc_id = json.loads(l)['doc']['id'].split('___')[0]
        if doc_id in selected_ids:
            out_file.write(l)

8171433it [01:28, 92193.97it/s] 


In [22]:
!cat {DIR}deep-ct-training-data-orcas-sampled-test-overlap-removed.jsonl|grep 'D1683937___5' | jq '.'

{
  "doc": {
    "title": "\nEach of our private cabanas are available for rent and include cool ceiling fans, refrigerators and comfortable chaise loungers. \nThe Hotel and Resort pools are available for use by Win\nStar World Casino Hotel guests only. \nThe hotel property also offers other leisurely amenities aside from poolside fun. \nStay in shape at the fitness center with our state-of-the-art exercise equipment. \nOr, make use of our two business centers to cover all of your business needs, so you can get back to relaxing. \nLearn More\nMeeting Rooms/Ballroom\nSpoil your guests with the hotel’s top-notch meeting facilities. \nYour corporate meeting, wedding, reunion or party is guaranteed to be a spectacular event in our 2,588 square foot versatile function space. \nPersonal sales and catering specialists are at your service to make sure your event is exactly the way you envisioned. \nOur meeting facilities consist of:\nAccommodating up to 100 and 150 guest for a cocktail social.

In [23]:
!cat {DIR}deep-ct-training-data-orcas-test-overlap-removed.jsonl|grep 'D1683937___5' | jq '.'

{
  "doc": {
    "title": "\nEach of our private cabanas are available for rent and include cool ceiling fans, refrigerators and comfortable chaise loungers. \nThe Hotel and Resort pools are available for use by Win\nStar World Casino Hotel guests only. \nThe hotel property also offers other leisurely amenities aside from poolside fun. \nStay in shape at the fitness center with our state-of-the-art exercise equipment. \nOr, make use of our two business centers to cover all of your business needs, so you can get back to relaxing. \nLearn More\nMeeting Rooms/Ballroom\nSpoil your guests with the hotel’s top-notch meeting facilities. \nYour corporate meeting, wedding, reunion or party is guaranteed to be a spectacular event in our 2,588 square foot versatile function space. \nPersonal sales and catering specialists are at your service to make sure your event is exactly the way you envisioned. \nOur meeting facilities consist of:\nAccommodating up to 100 and 150 guest for a cocktail social.